# Kas vyksta objekto sukūrimo metu?

Ši klasė turi specialų metodą, kurio vardas yra `__init__`. Šis metodas veikia panašiai, kaip konstruktoriai C++ ir Java kalbose. Konstruktoriai yra naudojami inicijuoti objekto būseną, t.y. priskirti reikšmes duomenų atstovams, kai klasės objektas yra sukuriamas. Kaip ir kiti metodai, konstruktorių taip pat sudaro komandos, vykdomos objekto kūrimo metu. Objektas sukuriamas baigus jas vykdyti. [[Versta iš GeeksForGeeks](https://www.geeksforgeeks.org/__init__-in-python/)]

Kodėl būtinas `__init__` metodo naudojimas? 

**Svarbu!** Dauguma klasės metodų argumentai prasideda nuo `self`. Tai reiškia, kad jie bus atliekami su `self` kintamuoju. Vardu `self` klasės metodo aprašyme įprastai žymimas tos klasės objekto egzempliorius.

Kai jau turime klasės `Polynomial` aprašą, galime sukurti pirmąjį tos klasės objektą, kurio vardas yra `p`. Kad būtų paprasčiau įsivaizduoti, galime sakyti, jog tai tam tikras daugianaris $P(x)= 2 - 7x + 5x^2$, priklausantis visų daugianarių aibei.

In [13]:
p = Expression([0, 1, 2], [2, -7, 5])

In [14]:
p.pows

[0, 1, 2]

In [ ]:
Šios komandos elgsena išorėje nematoma. Komandos viduje vykdymo metu atliekami tokie veiksmai:
    
* Sukuriamas egzempliorius `p`
* Automatiškai kviečiamas metodas `__init__(self, ...)`. Čia `self` atitinka kintamąjį, kuris yra 

In [ ]:
Pasiž

In [8]:
dir(p)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'coeffs',
 'kind',
 'pows']

In [5]:
print([func for func in dir(Polynomial) if callable(getattr(Polynomial, func))])

['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [ ]:
p.pows, p.coeffs #by default, jie priskiriami None reikšmėms

In [5]:
p = Polynomial()
p.pows, p.coeffs #by default, jie priskiriami None reikšmėms

(None, None)

Python priimta, kad kviečiant bet kurią klasę, ji iš pradžių kreipasi į `__init__` metodą, jei tokį randa. Šiame metode yra priskiriami atributai, kurie bus vartojami vėliau pagal poreikį.

In [25]:
P = Polynomial([0, 1, 2], [1, 2, 1]) #Polynomial kreipiasi į __init__ metodą, kuris priskiria pows ir coeffs atributus
P.pows, P.coeffs #Pasižiūrim šių atributų reikšmes

(array([0, 1]), array([1, 1]))

Įprastai kurti atributus, kuriems nėra peržiūros metodų, nėra gera programavimo praktika. Šiuo atveju metodus galėjome sukurti taip:

In [ ]:
    
    def get_pows(self):
        return self.pows
    
    def get_coeffs(self):
        return self.coeffs

In [16]:
class Expression:
    kind = 'polynomial'
    '''polynomial instance takes pows and coeffs
    >>> type(PolynomialRefactor(np.array([0,1]), np.array([1,1])))
    <class '__main__.PolynomialRefactor'>'''
    def __init__(self, pows=None, coeffs=None):
        self._pows = pows
        self._coeffs = coeffs
    
    def get_pows(self):
        '''return powers of polynomial
        >>> PolynomialRefactor(np.array([0,1]), np.array([1,1])).get_pows()
        array([0, 1])'''
        return self._pows
    
    def get_coeffs(self):
        '''return coeffs of polynomial
        >>> PolynomialRefactor(np.array([0,1]), np.array([1,1])).get_coeffs()
        array([1, 1])'''
        return self._coeffs